In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.integrate import solve_ivp
from scipy.linalg import solve,schur
from utility import orbit, BrusselatorModel, optim_BrusselatorModel, call_method
import sys
from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg import eigs
from scipy.interpolate import interp1d

from matplotlib.backends.backend_pdf import PdfPages
import time
from joblib import Parallel, delayed
import imageio
import cProfile
import pandas as pd

In [ ]:
if __name__ == "__main__":
    param_file = "./brusselator_params_2.in"  # JSON file containing model parameters
    model = BrusselatorModel(param_file)
    model_optim = optim_BrusselatorModel(param_file)
    print("Loaded parameters:", model_optim.n_z)

f = model.dydt
Jacf = model.brusselator_jacobian
# Parameters
z_L = model.z_L
# N = model.N
A, B = model.A, model.B
Max_iter = 7
epsilon = 1e-9 
T_0 = model.T_ini
# t_eval = np.linspace(0.0,20*model.T_ini, 1000)
# def solve_model(N, method):
#     model.N = N
#     z = np.linspace(0, z_L, N)
#     perturb = np.sin(np.pi*(z/z_L))

#     X0 = A + 0.01*perturb
#     Y0 = B/A + 0.01*perturb

#     y0 = np.concatenate([X0[1:-1],Y0[1:-1]])
#     # print('here')
#     start = time.time()
#     sol = solve_ivp(fun=f,t_span=[0.0, 10*T_0],
#                     t_eval=[0*10*model.T_ini], 
#                     dense_output = True,
#                     y0=y0, method= method, 
#                     **{"rtol": 1e-7,"atol":1e-9},
#                     )
#     end = time.time()
#     # y_T = sol.y[:,-1]
#     # y_T.shape
#     return sol, end - start



In [ ]:
# sol, t_solve = solve_model(30, "RK45")
# y0 = sol.y[:,-1]
# orbit_finder = orbit(f,y0,model.T_ini, Jacf,2, Max_iter, epsilon)

### Time execution check 

#### Newton

In [ ]:

model_optim.Lap

In [ ]:
# newton_time = []
# for N in np.arange(30,31,1):
model_optim.n_z = 120
model_optim.Lap = model_optim.Lap_mat()
f = model_optim.dydt
Jacf = model_optim.brusselator_jacobian

    
X0 = model_optim.A + 0.1*np.sin(np.pi*(np.linspace(0, model_optim.z_L, model_optim.n_z)/model_optim.z_L))
Y0 = model_optim.B/model_optim.A + 0.1*np.sin(np.pi*(np.linspace(0, model_optim.z_L, model_optim.n_z)/model_optim.z_L))
y0 = np.concatenate([X0[1:-1],Y0[1:-1]])
sol = solve_ivp(fun=f,t_span=[0.0, 16*model_optim.T_ini],
           t_eval=[16*model_optim.T_ini], 
            y0=y0, method='RK45', 
            **{"rtol": 1e-10,"atol":1e-12}
            )
y_T = sol.y[:,-1]
print(len(y0))
orbit_finder = orbit(f,y_T,model_optim.T_ini, Jacf,2, Max_iter, epsilon)
# start = time.time()
# profiler = cProfile.Profile()
# profiler.enable()
# k, T_by_iter, y_by_iter, Norm_B, Norm_Deltay = orbit_finder.Newton_orbit(y_T,model_optim.T_ini, Max_iter, epsilon)
# end = time.time()
# # newton_time.append(end - start)
# profiler.disable()
# profiler.print_stats(sort='time')
# T = T_by_iter[k-1]
# y = np.array(y_by_iter[k-1])
# print('T = ', T)
%prun -D Newton_optim_nz_120.prof orbit_finder.Newton_orbit(y_T,model_optim.T_ini, Max_iter, epsilon)

In [ ]:
model.n_z = 120
f = model.dydt
Jacf = model.brusselator_jacobian

orbit_finder = orbit(f,y_T,model.T_ini, Jacf,2, Max_iter, epsilon)
# start = time.time()
# k, T_by_iter, y_by_iter, Norm_B, Norm_Deltay = orbit_finder.Newton_orbit(y_T,model.T_ini, Max_iter, epsilon)
# end = time.time()
# # newton_time.append(end - start)
# profiler.disable()
# profiler.print_stats(sort='time')
# T = T_by_iter[k-1]
# y = np.array(y_by_iter[k-1])
# print('T = ', T)
%prun -D Newton_nz_120.prof orbit_finder.Newton_orbit(y_T,model_optim.T_ini, Max_iter, epsilon)


In [ ]:
model.n_z = 120
f = model.dydt
Jacf = model.brusselator_jacobian

orbit_finder = orbit(f,y_T,model.T_ini, Jacf,2, Max_iter, epsilon)
# start = time.time()
# k, T_by_iter, y_by_iter, Norm_B, Norm_Deltay = orbit_finder.Newton_orbit(y_T,model.T_ini, Max_iter, epsilon)
# end = time.time()
# # newton_time.append(end - start)
# profiler.disable()
# profiler.print_stats(sort='time')
# T = T_by_iter[k-1]
# y = np.array(y_by_iter[k-1])
# print('T = ', T)
%prun -D Newton_nz_120.prof orbit_finder.Newton_orbit(y_T,model_optim.T_ini, Max_iter, epsilon)

#### Newton-Picard with Simple subspace iteration

In [ ]:
p0, pe = 5 ,4
subspace_iter = 5
rho = 0.5
model.n_z = 32
# model.Lap = model.Lap_mat() #Upgrating the jacobian
f = model.dydt
Jacf = model.brusselator_jacobian
#Initialization
X0 = model.A + 0.1*np.sin(np.pi*(np.linspace(0, model.z_L, model.n_z)/model.z_L))
Y0 = model.B/model.A + 0.1*np.sin(np.pi*(np.linspace(0, model.z_L, model.n_z)/model.z_L))
y0 = np.concatenate([X0[1:-1],Y0[1:-1]])
#We integrate sufficiently the equation to find a good starting point
phi_t = solve_ivp(fun=f,t_span=[0.0, 16*model.T_ini],
            t_eval=[16*model.T_ini],
            dense_output=True,
            y0=y0, method='RK45', 
            **{"rtol": 1e-10,"atol":1e-12}
            )

y_T = phi_t.y[:,-1] #Using phi(y0,T0) as a starting point
orbit_finder = orbit(f,y_T,model.T_ini, Jacf,2, Max_iter, epsilon)
print('here')
V_0 = orbit_finder.subspace_iter(y_T,Ve_ini = np.eye(len(y_T))[:,:p0+pe],
                                 T =  model.T_ini,
                                 phi_t = phi_t,
                                 p0 = p0,
                                 pe = pe,
                                 max_iter = subspace_iter
                                 )

%prun -D NP_project_nz_32.prof orbit_finder.Newton_Picard_sub_proj(y_T,model.T_ini, Max_iter, epsilon,subspace_iter,V_0,p0,pe,rho)

model_optim.n_z = 32
model_optim.Lap = model_optim.Lap_mat()
f = model_optim.dydt
Jacf = model_optim.brusselator_jacobian
orbit_finder = orbit(f,y_T,model_optim.T_ini, Jacf,2, Max_iter, epsilon)
%prun -D NP_project_optim_nz_32.prof orbit_finder.Newton_Picard_sub_proj(y_T,model_optim.T_ini, Max_iter, epsilon,subspace_iter,V_0,p0,pe,rho)

In [ ]:
model.n_z = 120
# model.Lap = model.Lap_mat() #Upgrating the jacobian
f = model.dydt
Jacf = model.brusselator_jacobian
#Initialization
X0 = model.A + 0.1*np.sin(np.pi*(np.linspace(0, model.z_L, model.n_z)/model.z_L))
Y0 = model.B/model.A + 0.1*np.sin(np.pi*(np.linspace(0, model.z_L, model.n_z)/model.z_L))
y0 = np.concatenate([X0[1:-1],Y0[1:-1]])
#We integrate sufficiently the equation to find a good starting point
phi_t = solve_ivp(fun=f,t_span=[0.0, 16*model.T_ini],
            t_eval=[16*model.T_ini],
            dense_output=True,
            y0=y0, method='RK45', 
            **{"rtol": 1e-10,"atol":1e-12}
            )

y_T = phi_t.y[:,-1] #Using phi(y0,T0) as a starting point
orbit_finder = orbit(f,y_T,model.T_ini, Jacf,2, Max_iter, epsilon)
print('here')
V_0 = orbit_finder.subspace_iter(y_T,Ve_ini = np.eye(len(y_T))[:,:p0+pe],
                                 T =  model.T_ini,
                                 phi_t = phi_t,
                                 p0 = p0,
                                 pe = pe,
                                 max_iter = subspace_iter
                                 )

%prun -D NP_project_nz_120.prof orbit_finder.Newton_Picard_sub_proj(y_T,model.T_ini, Max_iter, epsilon,subspace_iter,V_0,p0,pe,rho)

model_optim.n_z = 120
model_optim.Lap = model_optim.Lap_mat()
f = model_optim.dydt
Jacf = model_optim.brusselator_jacobian
orbit_finder = orbit(f,y_T,model_optim.T_ini, Jacf,2, Max_iter, epsilon)
%prun -D NP_project_optim_nz_120.prof orbit_finder.Newton_Picard_sub_proj(y_T,model_optim.T_ini, Max_iter, epsilon,subspace_iter,V_0,p0,pe,rho)

In [ ]:
model.n_z = 200
# model.Lap = model.Lap_mat() #Upgrating the jacobian
f = model.dydt
Jacf = model.brusselator_jacobian
#Initialization
X0 = model.A + 0.1*np.sin(np.pi*(np.linspace(0, model.z_L, model.n_z)/model.z_L))
Y0 = model.B/model.A + 0.1*np.sin(np.pi*(np.linspace(0, model.z_L, model.n_z)/model.z_L))
y0 = np.concatenate([X0[1:-1],Y0[1:-1]])
#We integrate sufficiently the equation to find a good starting point
phi_t = solve_ivp(fun=f,t_span=[0.0, 16*model.T_ini],
            t_eval=[16*model.T_ini],
            dense_output=True,
            y0=y0, method='RK45', 
            **{"rtol": 1e-10,"atol":1e-12}
            )

y_T = phi_t.y[:,-1] #Using phi(y0,T0) as a starting point
orbit_finder = orbit(f,y_T,model.T_ini, Jacf,2, Max_iter, epsilon)
print('here')
V_0 = orbit_finder.subspace_iter(y_T,Ve_ini = np.eye(len(y_T))[:,:p0+pe],
                                 T =  model.T_ini,
                                 phi_t = phi_t,
                                 p0 = p0,
                                 pe = pe,
                                 max_iter = subspace_iter
                                 )

%prun -D NP_project_nz_200.prof orbit_finder.Newton_Picard_sub_proj(y_T,model.T_ini, Max_iter, epsilon,subspace_iter,V_0,p0,pe,rho)

model_optim.n_z = 200
model_optim.Lap = model_optim.Lap_mat()
f = model_optim.dydt
Jacf = model_optim.brusselator_jacobian
orbit_finder = orbit(f,y_T,model_optim.T_ini, Jacf,2, Max_iter, epsilon)
%prun -D NP_project_optim_nz_200.prof orbit_finder.Newton_Picard_sub_proj(y_T,model_optim.T_ini, Max_iter, epsilon,subspace_iter,V_0,p0,pe,rho)

In [ ]:
%%time
M@np.ones(N)

In [ ]:
%%time
A@np.ones(N)

In [ ]:
p0, pe = 5 ,4
subspace_iter = 5
rho = 0.5
# orbit_method = "Newton_Picard_simple"
def run(model, n_z,p0,pe,rho,Max_iter, subspace_iter,orbit_method):
    # df = pd.DataFrame()
    model.n_z = n_z
    model.Lap = model.Lap_mat() #Upgrating the jacobian
    f = model.dydt
    Jacf = model.brusselator_jacobian
    #Initialization
    X0 = model.A + 0.1*np.sin(np.pi*(np.linspace(0, model.z_L, model.n_z)/model.z_L))
    Y0 = model.B/model.A + 0.1*np.sin(np.pi*(np.linspace(0, model.z_L, model.n_z)/model.z_L))
    y0 = np.concatenate([X0[1:-1],Y0[1:-1]])
    #We integrate sufficiently the equation to find a good starting point
    phi_t = solve_ivp(fun=f,t_span=[0.0, 16*model.T_ini],
                t_eval=[16*model.T_ini],
                dense_output=True,
                y0=y0, method='RK45', 
                **{"rtol": 1e-10,"atol":1e-12}
                )
    
    y_T = phi_t.y[:,-1] #Using phi(y0,T0) as a starting point
    orbit_finder = orbit(f,y_T,model.T_ini, Jacf,2, Max_iter, epsilon)
    #Using the real part of the eigenvectors fo M(y_T,T0)
    #May be expensive in large dimension 
    # We may leverage in the fact that only the action of M is required while computing eigenvector rather than computing expicitely M
    # _,M = orbit_finder.integ_monodromy(y_T,model.T_ini)
    # _, eigvec = np.linalg.eig(M)
    # V_0 = np.real(eigvec[:,:p0+pe])
    # V_0,_ = np.linalg.qr(V_0)
    # _,V_0 = orbit_finder.base_Vp(np.eye(len(y_T))[0], y_T, model.T_ini, f, Jacf, p0+pe, epsilon)
    
    V_0 = orbit_finder.subspace_iter(y_T,Ve_ini = np.eye(len(y_T))[:,:p0+pe],
                                 T =  model.T_ini,
                                 phi_t = phi_t,
                                 p0 = p0,
                                 pe = pe,
                                 max_iter = subspace_iter
                                 )
    args = {
    "y0": y_T,
    "T_0": model.T_ini,
    "Max_iter": Max_iter,
    "epsilon": epsilon,
    "subsp_iter": subspace_iter,
    "Ve_0": V_0,
    "p0": p0,
    "pe": pe,
    "rho": rho,
    "phase_cond": 2}
    method_to_call = getattr(orbit_finder, orbit_method)

    start = time.time()
    k, T_by_iter, y_by_iter, Norm_B, Norm_Deltay = call_method(method_to_call, **args)
                         
    end = time.time()
    # p0 = p0+2 #We may vary p accordingly to N rather than fixing it
    results = dict(
        orbit_method = orbit_method,
        nz = n_z,
        p0 = p0,
        pe=pe,
        sub_sp_iter = subspace_iter,
        rho = rho,
        n_iter = k,
        precison = f"{epsilon:.1e}",
        n_ivp_solves = (subspace_iter*(p0+pe) + 1)*k,
        comput_time = float(f"{end-start:.5f}"),
        T_star = float(f"{T_by_iter[k-1]:.5f}"),
        T_iter = [T_by_iter],
    )
    # res = pd.DataFrame(results, index=[0])
    # df = pd.concat([df,res])
    # df.reset_index(drop=True)
    return results

In [ ]:
k=6
2 ** np.arange(4,4+k)

In [ ]:
n_z = 6
Max_iter = 20
orbit_method = "Newton_Picard_sub_proj"
# cProfile.run('run(model,N,p0,pe, rho,Max_iter,subspace_iter, orbit_method)')
profiler = cProfile.Profile()
profiler.enable()
run(model_optim,n_z,p0,pe, rho,Max_iter,subspace_iter, orbit_method)
profiler.disable()
profiler.print_stats(sort='time')

In [ ]:
orbit_method = "Newton_Picard_simple"
run(model,30,p0,pe, rho,Max_iter, subspace_iter, orbit_method)

In [ ]:
%%time
res1 = Parallel(n_jobs=2, prefer='processes',
               timeout=1000)(delayed(run)(model,N,p0,pe, rho,Max_iter, subspace_iter, orbit_method) for N in [20,30])

In [ ]:
pd.DataFrame(res1)

In [ ]:
from pathlib import Path
from datetime import datetime

BASE_PATH = Path().parent.resolve()
today_analysis = datetime.today().strftime('%Y-%m-%d_%H')
output_root_dir = BASE_PATH / "Results/"
 
Dir_path = Path(output_root_dir/today_analysis)
Dir_path.mkdir(parents=True, exist_ok=True)

In [ ]:
k=2
BASE_PATH/"brusselator_params_%i.in" %k

In [ ]:
file_path = f"{Dir_path/orbit_method}.txt"
file_path
#export DataFrame to text file
with open(file_path, 'w') as f:
    df_string = df.to_string()
    f.write(df_string)

In [ ]:
NP_simple_time = []
NP_simple_time.append(run(10,p0,pe))

In [ ]:
NP_simple_time = NP_simple_time + res1

#### Newton-Picard with subspace iteration with projection

In [ ]:
p0 = 5 
epsilon = 1e-9
Max_iter = 50
def run2(N,p0,pe,rho=0.2, subspace_iter = 4):
    model.N = N
    f = model.dydt
    Jacf = model.brusselator_jacobian
    print("P0 = ", p0)
    X0 = model.A + 0.1*np.sin(np.pi*(np.linspace(0, model.z_L, model.N)/model.z_L))
    Y0 = model.B/model.A + 0.1*np.sin(np.pi*(np.linspace(0, model.z_L, model.N)/model.z_L))
    y0 = np.concatenate([X0[1:-1],Y0[1:-1]])
    sol = solve_ivp(fun=f,t_span=[0.0, 20*model.T_ini],
                t_eval=[20*model.T_ini], 
                y0=y0, method='RK45', 
                **{"rtol": 1e-7,"atol":1e-9}
                )
    y_T = sol.y[:,-1]
    V_0 = np.eye(len(y_T))[:,:p0+pe]
    V_0,_ = np.linalg.qr(V_0)
    print("Dimension ", len(y0))
    orbit_finder = orbit(f,y_T,model.T_ini, Jacf,2, Max_iter, epsilon)
    start = time.time()
    k, T_by_iter, y_by_iter, Norm_B, Norm_Deltay = orbit_finder.NP_project_anim(f,y_T,
                         model.T_ini, V_0, p0,pe, rho, Jacf, Max_iter,subspace_iter, epsilon)
    end = time.time()
    p0 = p0+2

    return end - start


In [ ]:
run2(16,p0=25,pe=4)

In [ ]:
%%time
NP_projec = Parallel(n_jobs=8
               , prefer='processes',
               timeout=1000)(delayed(run2)(N,p0,pe) for N in np.arange(10,100,10)
               )

In [ ]:
Dim = np.arange(10,100,10)
fig, ax = plt.subplots(1,2,sharex='all',figsize=(12, 4))

ax[0].plot(Dim, NP_projec, label ="NP sub. projec")
ax[0].plot(Dim, NP_simple_time,'+-', label ="NP simple sub.")
ax[0].set_ylabel("Computational time in seconds")
ax[0].set_xlabel("Dimension N")
ax[0].legend()
ax[1].plot(Dim, newton_time, label ="Newton")
ax[1].set_ylabel("Computational time in seconds")
ax[1].set_xlabel("Dimension N")
ax[1].legend()

fig.subplots_adjust(left=0.09, bottom=0.1, right=0.95, top=0.90, hspace=0.35, wspace=0.35)

plt.savefig(f'./Results/brusslator/computational_time.png')



In [ ]:
np.eye(len(y_T))[0].shape

In [ ]:
p0, pe = 2 ,4
_,monodromy = orbit_finder.integ_monodromy(np.array(y_by_iter[k-1]),T_by_iter[k-1])
subspace_iter = 5
rho = 0.5
eig, eigvec = np.linalg.eig(monodromy)
V_0 = np.real(eigvec[:,:p0+pe])
V_0 = np.eye(len(y_T))[:,:p0+pe]
V_0,_ = np.linalg.qr(V_0)
v0 = V_0[:,0]
V_0.shape


### Checking the subspace iteration algorithm

In [ ]:
%%time
# eigen , Vp = orbit_finder.base_Vp(v0, y, T, f, Jacf, p0+pe, epsilon)
# Re, Ye, Ve, We,p =  orbit_finder.subsp_iter_projec(V_0, y, T, f, Jacf,rho, p0, pe, subspace_iter, tol=1e-6)
sol = solve_ivp(fun=f,t_span=[0.0, T],
                    t_eval=[T],
                    dense_output = True,
                    y0=y, method= "RK45", 
                    **{"rtol": 1e-7,"atol":1e-9}
                    )
# phi_t = interp1d(sol.t, sol.y, kind='linear', fill_value="extrapolate")
phi_t = sol
print('here')       


In [ ]:
%%time
Ve = orbit_finder.subspace_iter(V_0, T, Jacf,phi_t, p0, pe, subspace_iter)

In [ ]:
sol.y[:,-1] - sol.sol(T)

In [ ]:
We = np.column_stack([
                orbit_finder.monodromy_mult2( T, Jacf, Ve[:, j],
                                    phi_t)
                for j in range(Ve.shape[1])
            ])
Se = Ve.T @ We
Re, Ye, p= schur(Se, output='real',sort= lambda x,y: np.sqrt(x**2 + y**2) > rho)

In [ ]:
Mat_vect = LinearOperator((len(y0),len(y0)),matvec = lambda v : orbit_finder.monodromy_mult(y, T, f,Jacf, v, method = 2, epsilon = 1e-6))
Mat_vect2 = LinearOperator((len(y0),len(y0)),matvec = lambda v : orbit_finder.monodromy_mult2(T, Jacf, Ve[:, 0],phi_t))


In [ ]:
# np.linalg.norm(orbit_finder.monodromy_mult2(T, Jacf, Ve[:, 0],phi_t,
#                               ) - monodromy@Ve[:,0])
np.linalg.norm(Mat_vect2*Ve[:, 0] - monodromy@Ve[:,0])
# Mat_vect2*Ve[:, 0]

In [ ]:
# Vp,_ = np.linalg.qr(Ve @ Ye[:, :p])
Vp = Ve[:, :p]
# Vp = Ve @ Ye[:, :p]
# Vp,_ = np.linalg.qr(Ve @ Ye[:, :p])
print("Orthogonality :", np.allclose(Vp.T@Vp, np.eye(Vp.shape[1])))
np.linalg.norm(Vp[:,0],ord=2)


In [ ]:
eigen, _ = np.linalg.eig(Re)
p
# Ye[:,:p].shape

In [ ]:
mask1 = np.abs(eig) > rho
mask2 = np.abs(eigen) > rho
max_eigenvalues = np.sort(eig[mask1])[::-1]   
print("Eigenvalues from subspace iteration \n",(eigen[mask2]))
print("The first eigenvalues of the converged monodromy matrix \n", max_eigenvalues)
np.abs(eigen[mask2])

### Newton-Picard Gauss-Seidel 

In [ ]:
%%time
Max_iter = 30
p0, pe = 5,4
subsapace_iter = 5
# V_0 = np.real(eigvec[:,:p0+pe])
V_0 = np.eye(len(y_T))[:,:p0+pe]
k, T_by_iter, y_by_iter, Norm_B, Norm_Deltay = orbit_finder.Newton_Picard_subspace_iter(f,y_T,
                         T_0, V_0, p0,pe, rho, Jacf, Max_iter,subspace_iter, epsilon)

In [ ]:
Newton_time + Picard_time
# Picard_time

## Convergence check

In [ ]:
eig, eigvec = np.linalg.eig(monodromy)
# Extract real and imaginary parts of eigenvalues
real_parts = np.real(eig)
imaginary_parts = np.imag(eig)
print(imaginary_parts.shape)
# Create the figure and axis
fig, ax = plt.subplots(figsize=(6, 6))

# Plot the unit circle
theta = np.linspace(0, 2 * np.pi, 1000)
circle_x = np.cos(theta)
circle_y = np.sin(theta)
ax.plot(circle_x, circle_y, 'k--', label='Unit Circle')

# Plot the eigenvalues
ax.scatter(real_parts, imaginary_parts, color='r', label='Eigenvalues')

# Set labels and title
ax.set_xlabel(r'Re($\lambda$)')
ax.set_ylabel(r'Im($\lambda$)')
ax.set_title(f'Eigenvalues of the Monodromy matrix on Complex Plane \n with L = {model.L}')

# Set equal aspect ratio
ax.set_aspect('equal', 'box')

# Add grid, legend, and plot
ax.grid(True)
# ax.legend()

# Show the plot
plt.show()

Tab = np.asarray(y_by_iter[:k-1])
X = Tab[:,N-2:]
Y = Tab[:,:N-2]

fig0, ax = plt.subplots(2,2,sharex='all')
ax[0,0].plot(np.arange(k-1),X.mean(axis=1),'+-')
ax[0,0].set_ylabel(f"$<X>$")

ax[0,1].plot(np.arange(k-1),Y.mean(axis=1),'+-')
ax[0,1].set_ylabel(f"$<Y>$")
ax[1,0].semilogy(np.arange(k),Norm_Deltay[:k],'x-')
ax[1,0].set_xlabel("Newton iterations")
ax[1,0].set_ylabel(r"$\parallel \Delta y \parallel$")
ax[1,1].semilogy(np.arange(k),Norm_B[:k],'x-')
ax[1,1].set_xlabel("Newton iterations")
ax[1,1].set_ylabel(r"$\parallel \phi(y^*(0),T) - y^*(T) \parallel$")
# ax[1,1].set_ylabel(f"$\parallel (r,s) \parallel_2$")
fig0.set_size_inches((8,8))
fig0.suptitle(f'Brusselator model: $L=%.4f$ \n $T = %.4f$ ' % (model.L, T_by_iter[k-1]))
fig0.subplots_adjust(left=0.09, bottom=0.1, right=0.95, top=0.90, hspace=0.35, wspace=0.55)
# plt.savefig(f'./Results/brusselator/NP_test2_m_{str(m)}.png')

plt.show()
y0 = np.array(y_by_iter[k-1])
t_eval = np.linspace(0.0,2*T_by_iter[k-1], 1000)

sol = solve_ivp(fun=f,t_span=[0.0, 2*T_by_iter[k-1]],
                t_eval=t_eval, 
                y0=y0, method='RK45', 
                **{"rtol": 1e-7,"atol":1e-9}
                )

Xmean = sol.y[:N-2,:]
#Xmean = np.mean(Xmean,axis = 0)
Ymean = sol.y[N-2:,:]
#Ymean = np.mean(Ymean,axis = 0)
fig1, ax = plt.subplots(1,2)
ax[0].plot(t_eval, Xmean[23], label = '<X>')
ax[0].plot(t_eval, Ymean[23], label='<Y>')
ax[0].legend()
ax[0].set_ylabel("Concentrations")
ax[0].set_xlabel("t")
ax[1].plot(Xmean[23],Ymean[23])
ax[1].set_ylabel(r"$<Y>$")
ax[1].set_xlabel(r"$<X>$")
fig1.set_size_inches((10,5))
fig1.suptitle(r'Brusselator Model with Dirichlet BCs:')
fig1.subplots_adjust(left=0.09, bottom=0.1, right=0.95, top=0.90, hspace=0.35, wspace=0.55)
# plt.savefig(f'./Results/brusselator_1D.png')

#_____Saving the graphs into a pdf file__________
# ficout = model.out_dir + "test_NP_Arnoldi_%i.pdf" %model.num_test

# with PdfPages(ficout, keep_empty=False) as pdf:
#         pdf.savefig(figure=fig)
#         pdf.savefig(figure=fig0)
#         pdf.savefig(figure=fig1)

In [ ]:
%pip install imageio

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import imageio

images = []
for i in range(50):
    plt.figure()
    x = np.linspace(0, 2 * np.pi, 100)
    y = np.sin(x + i * 0.1)
    plt.plot(x, y)
    plt.title(f"Iteration {i}")
    plt.savefig(f"frame_{i}.png")
    plt.close()
    images.append(imageio.imread(f"frame_{i}.png"))

imageio.mimsave('iterative_animation.gif', images, duration=0.0005)

# You would typically delete the individual frame images after creating the animation
import os
for i in range(50):
    os.remove(f"frame_{i}.png")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import imageio
import os

def create_frame(i, color):
    """Creates a single frame with a specific plot color."""
    fig, ax = plt.subplots()
    ax.set_xlim(0, 2 * np.pi)
    ax.set_ylim(-1, 1)
    x = np.linspace(0, 2 * np.pi, 100)
    y = np.sin(x + i * 0.1)
    ax.plot(x, y, color=color)  # Set plot color here
    ax.set_title(f"Iteration {i}")

    filename = f"frame_{i}.png"
    plt.savefig(filename)
    plt.close(fig)
    return filename

if __name__ == "__main__":
    num_frames = 50
    plot_colors = ['blue', 'red', 'green', 'purple']
    filenames = []

    for i in range(num_frames):
        color_index = i % len(plot_colors)
        filename = create_frame(i, plot_colors[color_index])
        filenames.append(filename)

    # Read the generated frames
    images = [imageio.imread(filename) for filename in filenames]

    # Save as a GIF
    imageio.mimsave('alternating_plot_color.gif', images, duration=15.1)

    # Clean up individual frame files
    # for filename in filenames:
        # os.remove(filename)

    print("Animation 'alternating_plot_color.gif' created successfully!")